# 输出小部件：利用 Jupyter 的显示系统

在Jupyter环境中，`Output`小部件可以用于捕获和显示交互式计算的结果。通过将函数的输出重定向到`Output`小部件，你可以在用户界面中查看这些结果，而不是仅仅在代码单元格的输出区域中显示它们。这使得创建复杂的交互式应用程序成为可能，其中可以实时更新和共享计算结果。

以下是使用`Output`小部件的一些关键点：

1. **捕获输出**：你可以使用`display`函数将结果发送到`Output`小部件。例如，`display(some_output)`将`some_output`的值发送到当前的`Output`小部件。

2. **动态更新**：当你希望根据用户输入或其他事件动态更新输出时，`Output`小部件特别有用。你可以通过编程方式更新`Output`小部件的内容，以反映计算的最新状态。

3. **共享和保存**：`Output`小部件的内容可以轻松地共享给其他用户或保存为各种格式，如HTML、PDF或SVG。

4. **集成与自定义**：你可以将`Output`小部件与其他类型的小部件（如滑块、按钮等）结合使用，以创建具有丰富交互性的应用程序。此外，你还可以通过自定义CSS样式来控制`Output`小部件的外观。 

5. **灵活性**：`Output`小部件支持多种内容类型，包括文本、图像、视频等。这意味着你可以使用`Output`小部件来显示几乎任何类型的计算结果。

6. **扩展性**：如果你需要更高级的功能，可以创建自定义的`Output`小部件。这使你能够构建特定于你的应用程序或工作流程的专用输出控件。

总之，`Output` 小部件是 Jupyter 环境中的强大工具，它允许你以交互式和可视化的方式展示和共享计算结果。通过利用 `Output` 小部件，你可以创建更加动态和参与性的数据分析和科学计算应用程序。

In [1]:
# Imports for JupyterLite
%pip install -q ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ipywidgets as widgets

`Output` 部件可以捕获并显示 stdout、stderr 以及由 IPython 生成的 [富文本输出](http://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html#module-IPython.display)。你也可以直接将输出附加到输出部件，或者通过编程方式清除它。

In [3]:
out = widgets.Output(layout={'border': '1px solid black'})
out

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

在部件创建后，使用上下文管理器将输出定向到它。你可以将文本打印到输出区域：

In [4]:
with out:
    for i in range(10):
        print(i, 'Hello world!')

富文本输出也可以定向到输出区域。任何在 Jupyter 笔记本中显示良好的内容也会在 `Output` 部件中显示得很好。

```python
from IPython.display import YouTubeVideo
with out:
    display(YouTubeVideo('eWzY2nGfkXk'))
```

我们甚至可以在输出部件中显示复杂的 `mimetypes`，例如嵌套的部件。

In [5]:
with out:
    display(widgets.IntSlider())

我们还可以使用 `append_stdout`、`append_stderr` 或 `append_display_data` 等便利方法直接将输出附加到输出部件。

```python
out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('Output appended with append_stdout')
out.append_display_data(YouTubeVideo('eWzY2nGfkXk'))
out
```

可以通过在上下文管理器中使用 `IPython.display.clear_output` 来清除输出，或者我们可以直接调用部件的 `clear_output` 方法。

In [6]:
out.clear_output()

`clear_output` 支持关键字参数 `wait`。将其设置为 `True` 时，部件内容不会立即清除。相反，它们会在下一次部件接收到要显示的内容时被清除。在替换输出部件中的内容时，这很有用：它允许更平滑的过渡，避免了调用 `clear_output` 后部件突然调整大小的情况。

最后，我们可以使用输出部件通过 `capture` 装饰器捕获函数产生的所有输出。

In [7]:
@out.capture()
def function_with_captured_output():
    print('This goes into the output widget')
    raise Exception('As does this')
    
function_with_captured_output()

`out.capture` 支持关键字参数 `clear_output`。将其设置为 `True` 将在每次调用函数时清除输出部件，这样你只会看到最后一次调用的输出。将 `clear_output` 设置为 `True` 时，你还可以传递一个 `wait=True` 参数，以仅在新输出可用时清除输出。当然，你也可以随时手动清除输出。

In [8]:
out.clear_output()

## 输出部件作为交互的基础

输出部件构成了交互和相关方法实现的基础。它也可以单独使用，以创建包含部件和代码输出的丰富布局。自定义交互界面外观的一种简单方法是使用 `interactive_output` 函数将控件与一个函数连接起来，该函数的输出被捕获在返回的输出部件中。在下一个示例中，我们将控件垂直堆叠，然后将函数的输出放到右侧。

In [9]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

## 使用输出部件调试回调中的错误

在某些平台上，比如 JupyterLab，由部件回调生成的输出（例如，附加到部件特质的 `.observe` 方法或按钮部件的 `.on_click` 方法上的函数）不会在任何地方显示。即使在其他平台上，也不明确这些输出应该出现在哪个单元格中。这可能使得调试回调函数中的错误更具挑战性。

一个有效访问部件回调输出的工具是使用输出部件的捕获方法装饰回调函数。然后你可以在新的单元格中显示该部件以查看回调输出。

In [10]:
debug_view = widgets.Output(layout={'border': '1px solid black'})

@debug_view.capture(clear_output=True)
def bad_callback(event):
    print('This is about to explode')
    return 1.0 / 0.0

button = widgets.Button(
    description='click me to raise an exception',
    layout={'width': '300px'}
)
button.on_click(bad_callback)
button

Button(description='click me to raise an exception', layout=Layout(width='300px'), style=ButtonStyle())

In [11]:
debug_view

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

## 将输出部件与日志模块集成

虽然使用 `.capture` 装饰器对于理解和调试单个回调很有用，但它不适用于更大的应用程序。通常，在较大的应用程序中，人们可能会使用 [logging](https://docs.python.org/3/library/logging.html) 模块来打印程序状态的信息。然而，在部件应用程序的情况下，不清楚日志输出应该去哪里。

一个有用的模式是创建一个自定义的 [handler](https://docs.python.org/3/library/logging.html#handler-objects)，它将日志重定向到输出部件。然后可以在新的单元格中显示输出部件，以在应用程序运行时监视它。

In [12]:
import ipywidgets as widgets
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%', 
            'height': '160px', 
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout', 
            'output_type': 'stream', 
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs
        
    def show_logs(self):
        """ Show the logs """
        display(self.out)
    
    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [13]:
handler.show_logs()

Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

In [14]:
handler.clear_logs()
logger.info('Starting program')

try:
    logger.info('About to try something dangerous...')
    1.0/0.0
except Exception as e:
    logger.exception('An error occurred!')

## 从后台线程与输出部件交互

在显示由后台线程产生的输出时，Jupyter的 `display` 机制可能违反直觉。后台线程的输出被打印到主线程当前正在写入的任何单元格。要直接看到这一点，创建一个反复打印标准输出的线程：

```python
import threading
import time

def run():
    for i in itertools.count(0):
        time.sleep(1)
        print('output from background {}'.format(i))
        
t = threading.Thread(target=run)
t.start()
```

这总是在当前活动单元格中打印，而不是启动后台线程的单元格。

这可能导致输出部件出现意外行为。在输出部件捕获输出的时间内，笔记本中生成的任何输出，无论线程如何，都会进入输出部件。

避免意外的最佳方式是*永远*不要在多个线程生成输出的环境中使用输出部件的上下文管理器。相反，我们可以将输出部件传递给在线程中执行的函数，并使用 `append_display_data()`、`append_stdout()` 或 `append_stderr()` 方法将可显示的输出附加到输出部件。

In [15]:
import threading
from IPython.display import display, HTML
import ipywidgets as widgets
import time

def thread_func(something, out):
    for i in range(1, 5):
        time.sleep(0.3)
        out.append_stdout('{} {} {}\n'.format(i, '**'*i, something))
    out.append_display_data(HTML("<em>All done!</em>"))

display('Display in main thread')
out = widgets.Output()
# Now the key: the container is displayed (while empty) in the main thread
display(out)

thread = threading.Thread(
    target=thread_func,
    args=("some text", out))
thread.start()

'Display in main thread'

Output()

In [16]:
thread.join()